In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models

In [2]:
from tensorflow.keras.layers import Dense,Conv1D,Dropout,BatchNormalization,Input
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam,SGD,RMSprop,Adagrad

In [3]:
##Import of the keras tuner
# !pip install -U keras-tuner
import keras_tuner as kt

In [4]:
def build_cnn_lstm_model(hp):
    inputs = tf.keras.Input(shape=(24, 1))
    x = inputs

    # CNN part
    for i in range(hp.Int("num_conv_layers", min_value=1, max_value=8, step=2)):
        activation = hp.Choice(f"activation_conv_{i}", values=["relu", "tanh", "sigmoid"])
        kernel_size = hp.Int(f"kernel_size_{i}", min_value=2, max_value=5, step=1)
        filters = hp.Int(f"filters_{i}", min_value=16, max_value=64, step=16)
        x = layers.Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation=activation)(x)
        x = layers.MaxPooling1D(pool_size=2)(x)

    # LSTM part
    for i in range(hp.Int("num_lstm_layers", min_value=1, max_value=3, step=1)):
        units = hp.Int(f"lstm_units_{i}", min_value=32, max_value=128, step=32)
        activation = hp.Choice(f"activation_lstm_{i}", values=["relu", "tanh", "sigmoid"])
        x = layers.LSTM(units, return_sequences=True, activation=activation)(x)
        x = Dropout(0.2)(x)
        x = BatchNormalization()(x)

    # Final LSTM
    x = layers.LSTM(128, return_sequences=False, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    # Dense layers
    for i in range(hp.Int("num_dense_layers", min_value=1, max_value=8, step=1)):
        units = hp.Int(f"dense_units_{i}", min_value=64, max_value=256, step=64)
        activation = hp.Choice(f"activation_dense_{i}", values=["relu", "tanh", "sigmoid"])
        dropout_rate = hp.Float(f"dropout_rate_{i}", min_value=0.1, max_value=0.5, step=0.1)
        x = layers.Dense(units, activation=activation)(x)
        x = Dropout(dropout_rate)(x)
        x = BatchNormalization()(x)

    x = layers.Dense(512, activation="relu")(x)
    outputs = layers.Dense(1, activation="linear")(x)  # For regression

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    optimizer_choice = hp.Choice("optimizer", ["adam", "sgd", "rmsprop", "adagrad"])

    model.compile(
        optimizer=optimizer_choice,
        loss="mean_squared_error",
        metrics=["mae"]
    )
    return model

In [5]:
# X1=np.load("input.npy")
# y1=np.load("out.npy")

# X2=np.load("input2.npy")
# y2=np.load("out2.npy")

In [6]:
# 1d data
X1=np.load("input1_signal.npy")
y1=np.load("out1_signal.npy")

X2=np.load("input2_signal.npy")
y2=np.load("out2_signal.npy")


In [7]:
X1.shape

(1560, 24, 1)

In [8]:
X1[0].shape

(24, 1)

In [9]:
X1

array([[[-0.19905252],
        [-0.01478354],
        [ 0.40741432],
        ...,
        [ 2.00112371],
        [-0.79399977],
        [-3.24186447]],

       [[-4.73905293],
        [-4.95011526],
        [-3.93404466],
        ...,
        [ 1.82563351],
        [-0.54248442],
        [-2.99866096]],

       [[-4.93414706],
        [-5.78878389],
        [-5.24504844],
        ...,
        [ 1.29366792],
        [ 2.1795185 ],
        [ 2.50767461]],

       ...,

       [[ 1.06128226],
        [ 1.21483497],
        [ 1.11150744],
        ...,
        [-0.82496267],
        [-0.46555817],
        [ 0.01504179]],

       [[ 0.56620112],
        [ 1.19669901],
        [ 1.90761314],
        ...,
        [-3.26966915],
        [-1.27260231],
        [ 0.67268886]],

       [[ 2.26267697],
        [ 3.37018141],
        [ 4.05540859],
        ...,
        [-3.48018709],
        [-2.24342421],
        [-0.92209552]]])

In [11]:
## Final data
X=np.vstack([X1,X2])
y=np.vstack([y1,y2])

In [15]:
print(X.shape)
print(y.shape)
print(X.shape[:2])
print(y.shape[:1])

(4333, 24, 1)
(4333, 1)
(4333, 24)
(4333,)


In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)

In [17]:
##Tuner object
tuner=kt.RandomSearch(
    build_cnn_lstm_model,
    objective="val_mae",
    max_trials=10
)

In [19]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
tuner.get_best_hyperparameters()[0].values
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

# Save the best model
best_model.save("best_cnn_lstm_model.keras")